首先加载数据，可以看出训练集与测试集长度均为25000

In [1]:
import keras
import pandas_profiling
import numpy as np
import pandas as pd

imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
print(train_data[0])
print(len(train_data))
print(test_data[0])
print(len(test_data))

Using TensorFlow backend.


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
25000
[1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5, 4, 360,

将所有的影评对应的词id装换为影评原文，将所有标签转换为list格式，以便在在后面转换为DataFrame格式。

In [4]:
word_index = imdb.get_word_index()
word_index = {k: (v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3
reverse_word_index = {v: k for k, v in word_index.items()}


def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

train_text = []
for i in range(len(train_data)):
    text = decode_review(train_data[i])
    train_text.append(text)
print(train_text[0:10])

test_text = []
for i in range(len(test_data)):
    text = decode_review(test_data[i])
    test_text.append(text)
print(len(test_text))
train_label_list = list(train_labels)
test_label_list = list(test_labels)
print(train_label_list[0:10])

["<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for w

25000
[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]


这里由于panda_profiling这个数据可视化预览组件在windows无法使用，所以忽略以下错误

In [5]:
import threading
data_train1 = pd.DataFrame([train_text, train_label_list], index=['train', 'label'])
data_test1 = pd.DataFrame([test_text, test_label_list], index=['test', 'label'])

train = pandas_profiling.ProfileReport(data_train1)
    #test = pandas_profiling.ProfileReport(data_test1)
    
    #test.to_file(outputfile="test.html")


D:\Anaconda\Anaconda\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


MemoryError: 

将所有的数据转化为统一长度后，进行训练

In [6]:
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
process_train_x = pad_sequences(train_data, value=0, padding='post', maxlen=300)
process_test_x = pad_sequences(test_data, value=0, padding='post', maxlen=300)
train_x = process_train_x[:20000]
val_x = process_train_x[20000:]
train_y = train_labels[:20000]
val_y = train_labels[20000:]
model = Sequential()
model.add(Embedding(20000, 16))
model.add(GlobalAveragePooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.fit(train_x, train_y, epochs=30, batch_size=500,
          validation_data=[val_x, val_y])
res = model.evaluate(process_test_x, test_labels)
print(res)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          320000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 321,089
Trainable params: 321,089
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 20000 s